In [5]:
import tensorflow as tf
import numpy as np 
import cv2
import os
from tensorflow.keras.models import Sequential,load_model
from tensorflow.keras.layers import GlobalAveragePooling2D,Dropout,Convolution2D,Activation
from tensorflow.keras.utils import to_categorical




In [6]:
IMG_DIRECTORY = 'image_data'

CLASS_MAP = {
    "rock" : 0,
    "paper" : 1,
    "scissors" : 2,
    "none" : 3
}

NUM_CLASES = len(CLASS_MAP)

def mapper(val):
    return CLASS_MAP[val]


In [7]:
from tensorflow.keras.applications.densenet import DenseNet121

densent = DenseNet121(include_top=False, weights='imagenet',classes=4,input_shape=(227,227,3))

def get_model(base):
    
    model = Sequential()
    model.add(base)
    model.add(Dropout(0.5))
    model.add(Convolution2D(NUM_CLASES, (1,1), padding='valid'))
    model.add(Activation('relu'))
    model.add(GlobalAveragePooling2D())
    model.add(Activation('softmax'))

    return model




In [9]:
dataset = []

for dir in os.listdir(IMG_DIRECTORY):
    path = os.path.join(IMG_DIRECTORY, dir)
    if not os.path.isdir(path):
        continue

    for item in os.listdir(path):
        if item.startswith("."):
            continue
        img = cv2.imread(os.path.join(path, item))
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = cv2.resize(img, (227,227))
        
        dataset.append([img, dir])

print(dataset[5])


[array([[[254, 254, 247],
        [254, 254, 255],
        [252, 255, 255],
        ...,
        [252, 254, 253],
        [254, 255, 255],
        [253, 255, 254]],

       [[255, 254, 255],
        [ 74,  72,  94],
        [ 73,  74,  94],
        ...,
        [132, 134, 133],
        [132, 134, 133],
        [207, 208, 209]],

       [[253, 253, 255],
        [ 75,  73,  98],
        [ 75,  75, 102],
        ...,
        [133, 136, 137],
        [134, 137, 138],
        [207, 209, 210]],

       ...,

       [[255, 255, 255],
        [  0,   0,   0],
        [  0,   0,   0],
        ...,
        [  0,   0,   0],
        [  0,   0,   0],
        [158, 158, 158]],

       [[255, 255, 255],
        [  0,   0,   0],
        [  0,   0,   0],
        ...,
        [  0,   0,   0],
        [  0,   0,   0],
        [158, 158, 158]],

       [[255, 255, 255],
        [  0,   0,   0],
        [  0,   0,   0],
        ...,
        [  0,   0,   0],
        [  0,   0,   0],
        [158, 158, 158]

In [15]:
data, labels = zip(*dataset)
labels = list(map(mapper, labels))

labels = to_categorical(labels)

In [18]:
from tensorflow.keras.optimizers import Adam

dnet = get_model(densent)

dnet.compile(
    optimizer = Adam(lr=0.0001),
    loss='categorical_crossentropy',
    metrics=['acc']
)


In [19]:
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

np_train_data = np.array(data)
np_train_labels = np.array(labels)

checkpoint = ModelCheckpoint(
    'model.h5', 
    monitor='val_acc', 
    verbose=1, 
    save_best_only=True, 
    save_weights_only=True,
    mode='auto'
)

es = EarlyStopping(patience = 3)

dnet.fit(
    np_train_data,
    np_train_labels, 
    validation_split=0.2,
    verbose=2,
    epochs=10,
    callbacks=[checkpoint, es]
    )

dnet.save("model.h5")



Epoch 1/10


KeyboardInterrupt: 